Video Classification Model : https://towardsdatascience.com/transfer-learning-using-mobilenet-and-keras-c75daf7ff299

## STEP 1:  import the required libraries

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import cv2
from glob import glob
import os

from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


## STEP 2: Read each frame and their corresponding tag

In [2]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,A_OffSide_shot1.mp4_frame0.jpg,OffSide
1,A_OffSide_shot1.mp4_frame1.jpg,OffSide
2,A_OffSide_shot1.mp4_frame10.jpg,OffSide
3,A_OffSide_shot1.mp4_frame100.jpg,OffSide
4,A_OffSide_shot1.mp4_frame101.jpg,OffSide


## STEP 3: Read the frames and then store them as a NumPy array

In [3]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|████████████████████████████████████████████████████████████████████████████| 10991/10991 [04:02<00:00, 45.41it/s]


(10991, 224, 224, 3)

## STEP 4: Create the validation set

In [4]:
# separating the target
y = train['class']

In [5]:
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

## STEP 5: Create 4 different columns in the target, one for each category

In [6]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## STEP 6: Use the MobileNet pre-trained model to create the base model

Lets now re-use MobileNet as it is quite lightweight (17Mb), freeze the base layers and lets add and train the top few layers (https://machinethink.net/blog/compressing-deep-neural-nets/)

In [7]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = MobileNet(weights='imagenet',include_top=False) 


C:\Users\Jatin_Thakkar\AppData\Local\Continuum\anaconda3\envs\tensorflow_env\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


## STEP 7: Extract features from this pre-trained model for our training and validation images

In [8]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(8792, 7, 7, 1024)

In [9]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(2199, 7, 7, 1024)

## STEP 8: reshape the images into a single dimension

In [10]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(8792, 7*7*1024)
X_test = X_test.reshape(2199, 7*7*1024)

## STEP 9: Normalize the pixel values (between 0 and 1, helps the model to converge faster)

In [11]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

## STEP 10: Create the architecture of the model

In [12]:
# shape of images
X_train.shape

(8792, 50176)

In [13]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(50176,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## STEP 11: Train our model using the training frames and validate using validation frames

In [14]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight_MobileNet.hdf5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

Improving Generalization Performance by Switching from Adam to SGD
(https://towardsdatascience.com/normalized-direction-preserving-adam-switching-from-adam-to-sgd-and-nesterov-momentum-adam-with-460be5ddf686)

In [15]:
# compiling the model
from keras.optimizers import SGD
opt = SGD()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              51381248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
__________

In [16]:
# training the model
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8792 samples, validate on 2199 samples
Epoch 1/200
8792/8792 [==============================] - ETA: 4:04 - loss: 1.6181 - acc: 0.351 - ETA: 2:15 - loss: 1.5556 - acc: 0.343 - ETA: 1:39 - loss: 1.5181 - acc: 0.351 - ETA: 1:21 - loss: 1.4912 - acc: 0.371 - ETA: 1:10 - loss: 1.4506 - acc: 0.392 - ETA: 1:02 - loss: 1.4391 - acc: 0.398 - ETA: 56s - loss: 1.4163 - acc: 0.410 - ETA: 52s - loss: 1.4020 - acc: 0.41 - ETA: 48s - loss: 1.3876 - acc: 0.41 - ETA: 45s - loss: 1.3717 - acc: 0.42 - ETA: 43s - loss: 1.3620 - acc: 0.42 - ETA: 41s - loss: 1.3407 - acc: 0.44 - ETA: 39s - loss: 1.3432 - acc: 0.44 - ETA: 37s - loss: 1.3364 - acc: 0.44 - ETA: 35s - loss: 1.3284 - acc: 0.44 - ETA: 34s - loss: 1.3264 - acc: 0.44 - ETA: 33s - loss: 1.3166 - acc: 0.44 - ETA: 31s - loss: 1.3111 - acc: 0.45 - ETA: 30s - loss: 1.3022 - acc: 0.45 - ETA: 30s - loss: 1.3029 - acc: 0.45 - ETA: 29s - loss: 1.3032 - ac

8792/8792 [==============================] - ETA: 39s - loss: 1.1807 - acc: 0.46 - ETA: 36s - loss: 1.1262 - acc: 0.48 - ETA: 34s - loss: 1.0964 - acc: 0.48 - ETA: 34s - loss: 1.0816 - acc: 0.49 - ETA: 38s - loss: 1.0577 - acc: 0.51 - ETA: 38s - loss: 1.0444 - acc: 0.52 - ETA: 36s - loss: 1.0290 - acc: 0.53 - ETA: 35s - loss: 1.0271 - acc: 0.54 - ETA: 34s - loss: 1.0242 - acc: 0.54 - ETA: 33s - loss: 1.0180 - acc: 0.54 - ETA: 32s - loss: 1.0094 - acc: 0.55 - ETA: 32s - loss: 1.0107 - acc: 0.55 - ETA: 31s - loss: 1.0129 - acc: 0.55 - ETA: 30s - loss: 1.0106 - acc: 0.55 - ETA: 29s - loss: 1.0085 - acc: 0.55 - ETA: 29s - loss: 1.0084 - acc: 0.55 - ETA: 28s - loss: 1.0065 - acc: 0.55 - ETA: 27s - loss: 1.0073 - acc: 0.55 - ETA: 26s - loss: 1.0098 - acc: 0.55 - ETA: 25s - loss: 1.0051 - acc: 0.56 - ETA: 25s - loss: 0.9983 - acc: 0.56 - ETA: 24s - loss: 0.9998 - acc: 0.56 - ETA: 24s - loss: 1.0056 - acc: 0.56 - ETA: 23s - loss: 1.0039 - acc: 0.56 - ETA: 23s - loss: 1.0046 - acc: 0.56 - ETA: 

8792/8792 [==============================] - ETA: 31s - loss: 0.9760 - acc: 0.54 - ETA: 30s - loss: 0.9775 - acc: 0.55 - ETA: 30s - loss: 0.9697 - acc: 0.55 - ETA: 30s - loss: 0.9635 - acc: 0.55 - ETA: 30s - loss: 0.9480 - acc: 0.55 - ETA: 29s - loss: 0.9399 - acc: 0.56 - ETA: 28s - loss: 0.9228 - acc: 0.57 - ETA: 27s - loss: 0.9173 - acc: 0.58 - ETA: 27s - loss: 0.9301 - acc: 0.57 - ETA: 27s - loss: 0.9383 - acc: 0.57 - ETA: 26s - loss: 0.9400 - acc: 0.57 - ETA: 26s - loss: 0.9388 - acc: 0.56 - ETA: 26s - loss: 0.9321 - acc: 0.57 - ETA: 25s - loss: 0.9315 - acc: 0.57 - ETA: 25s - loss: 0.9274 - acc: 0.57 - ETA: 25s - loss: 0.9261 - acc: 0.57 - ETA: 25s - loss: 0.9187 - acc: 0.57 - ETA: 24s - loss: 0.9158 - acc: 0.57 - ETA: 24s - loss: 0.9185 - acc: 0.57 - ETA: 23s - loss: 0.9159 - acc: 0.58 - ETA: 23s - loss: 0.9153 - acc: 0.58 - ETA: 22s - loss: 0.9137 - acc: 0.58 - ETA: 22s - loss: 0.9135 - acc: 0.58 - ETA: 21s - loss: 0.9144 - acc: 0.58 - ETA: 22s - loss: 0.9160 - acc: 0.58 - ETA: 

8792/8792 [==============================] - ETA: 27s - loss: 0.8081 - acc: 0.65 - ETA: 28s - loss: 0.7749 - acc: 0.68 - ETA: 27s - loss: 0.7789 - acc: 0.67 - ETA: 27s - loss: 0.7862 - acc: 0.66 - ETA: 27s - loss: 0.7728 - acc: 0.66 - ETA: 27s - loss: 0.7839 - acc: 0.65 - ETA: 26s - loss: 0.7867 - acc: 0.64 - ETA: 26s - loss: 0.7896 - acc: 0.65 - ETA: 25s - loss: 0.7930 - acc: 0.65 - ETA: 25s - loss: 0.7853 - acc: 0.65 - ETA: 25s - loss: 0.7763 - acc: 0.66 - ETA: 24s - loss: 0.7733 - acc: 0.65 - ETA: 24s - loss: 0.7677 - acc: 0.66 - ETA: 23s - loss: 0.7649 - acc: 0.66 - ETA: 23s - loss: 0.7650 - acc: 0.66 - ETA: 22s - loss: 0.7649 - acc: 0.66 - ETA: 22s - loss: 0.7607 - acc: 0.66 - ETA: 21s - loss: 0.7553 - acc: 0.66 - ETA: 21s - loss: 0.7585 - acc: 0.66 - ETA: 20s - loss: 0.7595 - acc: 0.66 - ETA: 20s - loss: 0.7556 - acc: 0.67 - ETA: 19s - loss: 0.7547 - acc: 0.67 - ETA: 19s - loss: 0.7544 - acc: 0.67 - ETA: 18s - loss: 0.7568 - acc: 0.67 - ETA: 18s - loss: 0.7591 - acc: 0.67 - ETA: 

8792/8792 [==============================] - ETA: 26s - loss: 0.5051 - acc: 0.84 - ETA: 26s - loss: 0.5366 - acc: 0.80 - ETA: 26s - loss: 0.5679 - acc: 0.77 - ETA: 25s - loss: 0.5748 - acc: 0.76 - ETA: 25s - loss: 0.5883 - acc: 0.77 - ETA: 24s - loss: 0.6206 - acc: 0.75 - ETA: 24s - loss: 0.6368 - acc: 0.74 - ETA: 24s - loss: 0.6283 - acc: 0.74 - ETA: 23s - loss: 0.6174 - acc: 0.74 - ETA: 23s - loss: 0.6090 - acc: 0.75 - ETA: 23s - loss: 0.6012 - acc: 0.75 - ETA: 22s - loss: 0.5951 - acc: 0.75 - ETA: 22s - loss: 0.5919 - acc: 0.75 - ETA: 21s - loss: 0.5892 - acc: 0.76 - ETA: 21s - loss: 0.5909 - acc: 0.75 - ETA: 21s - loss: 0.5940 - acc: 0.75 - ETA: 20s - loss: 0.5914 - acc: 0.75 - ETA: 20s - loss: 0.5894 - acc: 0.76 - ETA: 19s - loss: 0.5851 - acc: 0.76 - ETA: 19s - loss: 0.5825 - acc: 0.76 - ETA: 19s - loss: 0.5791 - acc: 0.76 - ETA: 18s - loss: 0.5770 - acc: 0.76 - ETA: 18s - loss: 0.5831 - acc: 0.76 - ETA: 17s - loss: 0.5835 - acc: 0.76 - ETA: 17s - loss: 0.5853 - acc: 0.76 - ETA: 

...


Epoch 00190: val_loss did not improve from 0.00000
Epoch 191/200
8792/8792 [==============================] - ETA: 27s - loss: 0.0036 - acc: 1.00 - ETA: 30s - loss: 0.0031 - acc: 1.00 - ETA: 29s - loss: 0.0025 - acc: 1.00 - ETA: 28s - loss: 0.0027 - acc: 1.00 - ETA: 28s - loss: 0.0023 - acc: 1.00 - ETA: 27s - loss: 0.0022 - acc: 1.00 - ETA: 26s - loss: 0.0085 - acc: 0.99 - ETA: 26s - loss: 0.0095 - acc: 0.99 - ETA: 25s - loss: 0.0085 - acc: 0.99 - ETA: 25s - loss: 0.0077 - acc: 0.99 - ETA: 24s - loss: 0.0083 - acc: 0.99 - ETA: 24s - loss: 0.0078 - acc: 0.99 - ETA: 23s - loss: 0.0073 - acc: 0.99 - ETA: 23s - loss: 0.0069 - acc: 0.99 - ETA: 22s - loss: 0.0068 - acc: 0.99 - ETA: 22s - loss: 0.0064 - acc: 0.99 - ETA: 21s - loss: 0.0060 - acc: 0.99 - ETA: 21s - loss: 0.0060 - acc: 0.99 - ETA: 20s - loss: 0.0057 - acc: 0.99 - ETA: 20s - loss: 0.0055 - acc: 0.99 - ETA: 20s - loss: 0.0054 - acc: 0.99 - ETA: 19s - loss: 0.0053 - acc: 0.99 - ETA: 19s - loss: 0.0051 - acc: 0.99 - ETA: 18s - loss

8792/8792 [==============================] - ETA: 27s - loss: 0.0011 - acc: 1.00 - ETA: 26s - loss: 0.0011 - acc: 1.00 - ETA: 26s - loss: 0.0097 - acc: 0.99 - ETA: 26s - loss: 0.0078 - acc: 0.99 - ETA: 26s - loss: 0.0066 - acc: 0.99 - ETA: 26s - loss: 0.0057 - acc: 0.99 - ETA: 25s - loss: 0.0050 - acc: 0.99 - ETA: 25s - loss: 0.0046 - acc: 0.99 - ETA: 25s - loss: 0.0045 - acc: 0.99 - ETA: 24s - loss: 0.0044 - acc: 0.99 - ETA: 24s - loss: 0.0043 - acc: 0.99 - ETA: 24s - loss: 0.0039 - acc: 0.99 - ETA: 23s - loss: 0.0037 - acc: 0.99 - ETA: 23s - loss: 0.0036 - acc: 0.99 - ETA: 22s - loss: 0.0035 - acc: 0.99 - ETA: 22s - loss: 0.0033 - acc: 0.99 - ETA: 21s - loss: 0.0032 - acc: 0.99 - ETA: 21s - loss: 0.0031 - acc: 0.99 - ETA: 21s - loss: 0.0038 - acc: 0.99 - ETA: 20s - loss: 0.0038 - acc: 0.99 - ETA: 20s - loss: 0.0037 - acc: 0.99 - ETA: 20s - loss: 0.0037 - acc: 0.99 - ETA: 20s - loss: 0.0036 - acc: 0.99 - ETA: 20s - loss: 0.0035 - acc: 0.99 - ETA: 19s - loss: 0.0034 - acc: 0.99 - ETA: 

8792/8792 [==============================] - ETA: 27s - loss: 0.0012 - acc: 1.00 - ETA: 27s - loss: 0.0013 - acc: 1.00 - ETA: 27s - loss: 0.0022 - acc: 1.00 - ETA: 26s - loss: 0.0018 - acc: 1.00 - ETA: 26s - loss: 0.0016 - acc: 1.00 - ETA: 26s - loss: 0.0016 - acc: 1.00 - ETA: 25s - loss: 0.0018 - acc: 1.00 - ETA: 25s - loss: 0.0018 - acc: 1.00 - ETA: 24s - loss: 0.0018 - acc: 1.00 - ETA: 24s - loss: 0.0020 - acc: 1.00 - ETA: 24s - loss: 0.0019 - acc: 1.00 - ETA: 24s - loss: 0.0019 - acc: 1.00 - ETA: 23s - loss: 0.0018 - acc: 1.00 - ETA: 23s - loss: 0.0017 - acc: 1.00 - ETA: 22s - loss: 0.0016 - acc: 1.00 - ETA: 22s - loss: 0.0015 - acc: 1.00 - ETA: 22s - loss: 0.0016 - acc: 1.00 - ETA: 21s - loss: 0.0016 - acc: 1.00 - ETA: 21s - loss: 0.0016 - acc: 1.00 - ETA: 20s - loss: 0.0016 - acc: 1.00 - ETA: 20s - loss: 0.0017 - acc: 1.00 - ETA: 20s - loss: 0.0016 - acc: 1.00 - ETA: 19s - loss: 0.0017 - acc: 1.00 - ETA: 19s - loss: 0.0019 - acc: 0.99 - ETA: 18s - loss: 0.0019 - acc: 0.99 - ETA: 

8792/8792 [==============================] - ETA: 30s - loss: 1.5156e-04 - acc: 1.00 - ETA: 28s - loss: 2.7176e-04 - acc: 1.00 - ETA: 27s - loss: 2.8391e-04 - acc: 1.00 - ETA: 27s - loss: 3.6139e-04 - acc: 1.00 - ETA: 27s - loss: 6.6548e-04 - acc: 1.00 - ETA: 26s - loss: 8.4415e-04 - acc: 1.00 - ETA: 26s - loss: 9.8274e-04 - acc: 1.00 - ETA: 25s - loss: 0.0011 - acc: 1.0000   - ETA: 25s - loss: 9.8315e-04 - acc: 1.00 - ETA: 24s - loss: 0.0012 - acc: 1.0000   - ETA: 24s - loss: 0.0013 - acc: 1.00 - ETA: 23s - loss: 0.0012 - acc: 1.00 - ETA: 23s - loss: 0.0012 - acc: 1.00 - ETA: 23s - loss: 0.0012 - acc: 1.00 - ETA: 22s - loss: 0.0011 - acc: 1.00 - ETA: 22s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 22s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 21s - loss: 0.0012 - acc: 1.00 - ETA: 20s - loss: 0.0012 - acc: 1.00 - ETA

8792/8792 [==============================] - ETA: 28s - loss: 6.8958e-05 - acc: 1.00 - ETA: 27s - loss: 0.0014 - acc: 1.0000   - ETA: 26s - loss: 0.0011 - acc: 1.00 - ETA: 26s - loss: 0.0042 - acc: 0.99 - ETA: 26s - loss: 0.0036 - acc: 0.99 - ETA: 26s - loss: 0.0051 - acc: 0.99 - ETA: 25s - loss: 0.0079 - acc: 0.99 - ETA: 25s - loss: 0.0071 - acc: 0.99 - ETA: 25s - loss: 0.0064 - acc: 0.99 - ETA: 24s - loss: 0.0058 - acc: 0.99 - ETA: 27s - loss: 0.0053 - acc: 0.99 - ETA: 28s - loss: 0.0051 - acc: 0.99 - ETA: 27s - loss: 0.0051 - acc: 0.99 - ETA: 26s - loss: 0.0048 - acc: 0.99 - ETA: 26s - loss: 0.0049 - acc: 0.99 - ETA: 25s - loss: 0.0047 - acc: 0.99 - ETA: 25s - loss: 0.0056 - acc: 0.99 - ETA: 25s - loss: 0.0053 - acc: 0.99 - ETA: 24s - loss: 0.0052 - acc: 0.99 - ETA: 24s - loss: 0.0052 - acc: 0.99 - ETA: 23s - loss: 0.0051 - acc: 0.99 - ETA: 23s - loss: 0.0050 - acc: 0.99 - ETA: 22s - loss: 0.0048 - acc: 0.99 - ETA: 22s - loss: 0.0047 - acc: 0.99 - ETA: 21s - loss: 0.0046 - acc: 0.99